In [1]:
from keras.models import model_from_json
import os
import cv2
import glob
import h5py
# import pandas as pd
from sklearn.metrics import mean_absolute_error
import scipy.io as io
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras

Using TensorFlow backend.
C:\Users\asus\anaconda3\envs\csrnet_kb\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\asus\anaconda3\envs\csrnet_kb\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\asus\anaconda3\envs\csrnet_kb\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\asus\anaconda3\envs\csrnet_kb

In [2]:
# with open('models/SANet.json', 'r') as json_file:
#     json_savedModel= json_file.read()
# model = tf.keras.models.model_from_json(json_savedModel)
# model.summary()

In [3]:
# weights_path = "weights_B_ssim_eucli_loss_bestMAE27.467_Sun-Mar-19/SANet_best.hdf5"
# model.load_weights(weights_path)

In [4]:
def load_model(option):
    
    json_file = open('models/DD_SCN.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    if(option=='A'):
        loaded_model.load_weights(("weights_A_ssim_eucli_loss_bestMAE215.379_Mon-Apr-24/DD_SCN_best.hdf5"))
    if(option=='B'):
        loaded_model.load_weights(("weights_B_ssim_eucli_loss_bestMAE38.028_Mon-Apr-24/DD_SCN_best.hdf5"))
    return loaded_model


def create_img(path):
    im = Image.open(path).convert('RGB')
    
    im = np.array(im)
    
    im = im/255.0
    
    im[:,:,0]=(im[:,:,0]-0.485)/0.229
    im[:,:,1]=(im[:,:,1]-0.456)/0.224
    im[:,:,2]=(im[:,:,2]-0.406)/0.225


    im = np.expand_dims(im,axis  = 0)
    return im

In [5]:
root = 'data/ShanghaiTech'
part_A_train = os.path.join(root,'part_A/train_data','images')
part_A_test = os.path.join(root,'part_A/test_data','images')
part_B_train = os.path.join(root,'part_B/train_data','images')
part_B_test = os.path.join(root,'part_B/test_data','images')


In [6]:
path_sets = [part_A_test]

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
# print(len(img_paths))

In [7]:
model = load_model('A')
name = []
y_true = []
y_pred = []


for image in img_paths:
    name.append(image)
    gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground') )
    groundtruth = np.asarray(gt['density'])
    num1 = np.sum(groundtruth)
    y_true.append(np.sum(num1))
    img = create_img(image)
    num = np.sum(model.predict(img))
    y_pred.append(np.sum(num))

    
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))
print("MAE Part A: " , ans )

C:\Users\asus\anaconda3\envs\csrnet_kb\lib\site-packages\ipykernel_launcher.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  if __name__ == '__main__':


MAE Part A:  216.24414


In [8]:
path_sets = [part_B_test]

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)
# print(len(img_paths))

In [9]:
model = load_model('B')
name = []
y_true = []
y_pred = []


for image in img_paths:
    name.append(image)
    gt = h5py.File(image.replace('.jpg','.h5').replace('images','ground') )
    groundtruth = np.asarray(gt['density'])
    num1 = np.sum(groundtruth)
    y_true.append(np.sum(num1))
    img = create_img(image)
    num = np.sum(model.predict(img))
    y_pred.append(np.sum(num))

    
ans = mean_absolute_error(np.array(y_true),np.array(y_pred))
print("MAE Part B: " , ans )

C:\Users\asus\anaconda3\envs\csrnet_kb\lib\site-packages\ipykernel_launcher.py:9: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  if __name__ == '__main__':


MAE Part B:  38.027897
